In [27]:
import pandas as pd
import numpy as np
import os
import requests

In [43]:
# pip install mysql-connector-python

In [28]:
import pyodbc
import urllib
import sqlalchemy
# MySQL connection parameters
db_params = {
    'user': 'han',
    'password': 'joseph',
    'host': '127.0.0.1',  # Usually 'localhost' or an IP address
    'port': '4450',  # Default MySQL port is 3306
    'database': 'ITRAVEL',
}

# URL-encode the password (if it contains special characters)
db_params['password'] = urllib.parse.quote_plus(db_params['password'])

# Create a MySQL database connection using SQLAlchemy
params_mysql = "mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}".format(**db_params)
engine_mysql = sqlalchemy.create_engine(params_mysql)

In [29]:
engine_p=engine_mysql

In [ ]:
'''get locations data from source database'''
df_location_existing = pd.read_sql_table('locations',engine_p)
# df_location_existing.to_sql("locations_back",engine_mysql,if_exists='replace',index=False)
df_location_existing.info()

In [108]:
# df_location_existing=df_location_existing.drop_duplicates(subset='location_id')
# df_location_existing.info()

# df_location_existing.to_sql("locations",engine_mysql,if_exists='replace',index=False)

df_location_existing

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4288 entries, 0 to 4567
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           4288 non-null   int64 
 1   location_id  4288 non-null   int64 
 2   name         4288 non-null   object
 3   distance     4288 non-null   object
 4   bearing      4288 non-null   object
 5   address_obj  4288 non-null   object
dtypes: int64(2), object(4)
memory usage: 234.5+ KB


,id,location_id,name,distance,bearing,address_obj
0,1,15706617,Bib - English Breakfast in the Box,9.540954320756208,southwest,"{'street1': 'Lower Ground Level, Al Shafar Tow..."
1,2,10165445,Ayamna,10.884259167826489,southwest,"{'street1': 'Crescent road', 'street2': 'The P..."
2,3,1022759,Atlantis The Palm,10.907456424513168,southwest,"{'street1': 'Crescent Road', 'city': 'Dubai', ..."
3,4,17836852,Tulum Restaurant,0.6873599138082985,southeast,"{'street1': 'The Dubai Mall Unit No. 4F - 04',..."
4,5,942786,Asado Restaurant,0.8196275775014424,southeast,"{'street1': 'Shaik Rashid Blvd', 'street2': 'T..."
...,...,...,...,...,...,...
4563,4564,17586896,Pizza Hut,30.45022458738102,south,"{'street1': 'ENOC Petrol Station, Emirates Roa..."
4564,4565,17782642,Pizza Express,92.60937210040943,south,"{'street1': 'Othaman Bin Affan Street', 'stree..."
4565,4566,911410,Japengo,28.055398871079547,southwest,"{'street1': 'Palm Strip shopping mall', 'stree..."
4566,4567,17721128,Kubyertos,43.43651715662099,southwest,"{'street1': 'Discovery Gardens Pavilion, Stree..."


In [ ]:
Dubai	25.263056, 55.297222
Abu Dhabi	24.466667, 54.366667
Sharjah		25.3575, 55.390833
Al Ain	24.2075, 55.744722
Ajman	25.413611, 55.445556
Ras Al Khaimah	25.783333, 55.95
Fujairah	25.121927, 56.346876
Umm Al Quwain	25.544095, 55.553305
Kalba	25.074167, 56.355278
Madinat Zayed	23.652222, 53.653611
Khor Fakkan		25.333333, 56.35
Dibba Al-Fujairah	25.591, 56.26
Ruwais	24.103333, 52.583611
Ghayathi	23.8425, 52.81
Dhaid	25.283333, 55.883333
Jebel Ali	25.01126, 55.06116
Liwa Oasis	23.133333, 53.766667
Hatta	24.796667, 56.1175
Ar-Rams		25.878889, 56.023611
Dibba Al-Hisn	25.618889, 56.273333
Al Jazirah Al Hamra		25.6832, 55.8200


In [6]:
# Define a location reviews function
def call_locations(location,coordinates):
    """This function get location reviews through api ."""
    url_pattern = coordinates.replace(",", "%2C")
    
    # API URL
#     api_url = "https://api.content.tripadvisor.com/api/v1/location/search?searchQuery=" + str(location) + "&latLong=25.2048%2C%2055.2708&language=en&key=72577774A72F48D9BF1180DEA12E312A"
#     api_url = "https://api.content.tripadvisor.com/api/v1/location/search?searchQuery=" + str(location) + "&latLong=24.466667%2C%2054.366667&language=en&key=72577774A72F48D9BF1180DEA12E312A"
        
    api_url = "https://api.content.tripadvisor.com/api/v1/location/search?searchQuery=" + urllib.parse.quote(str(location)) + "&latLong=" +  urllib.parse.quote(str(coordinates)) + "&language=en&key=72577774A72F48D9BF1180DEA12E312A"


    # Make API request
    response = requests.get(api_url)
    data = response.json()
    
    return data 

In [7]:
# Define a location data data frame
def get_df_locations(non_empty_strings,coordinates):
    # Loop through DataFrame rows
    df_locations_new=pd.DataFrame()

    for item in non_empty_strings:
        location = item
        
        data=call_locations(location,coordinates)
#     df1 = pd.DataFrame(data['data'])
        try:
            df1 = pd.DataFrame(data['data'])
        except KeyError:
            print("Key 'data' not found in the JSON response.")
            df1 = pd.DataFrame() 
    
        df_locations_new=pd.concat([df_locations_new, df1], ignore_index=True)
    
    return df_locations_new


In [26]:
import re

directory_path = r"F:\itravel\data\rest_local\1"

# Get a list of all files in the directory
file_paths = os.listdir(directory_path)

data=''
# Initialize a list to store the 2-dimensional array
two_dimensional_array = []

# Loop through the file paths and read each file into a DataFrame
for file_path in file_paths:
    # file_path = 'C:/Users/pc/Downloads/restaurant_16'

    # Open the file in read mode ('r')
    with open(directory_path+'\\'+file_path, 'r', encoding='utf-8') as file:

        # Read the entire file content into a string
        file_name = os.path.basename(file_path)
        parts = file_name.split('_')

        # Get the last part
        last_part = parts[-1]
        coordinates = last_part
        
        data = file.read()
        
        # Use regular expressions to find lines that start with a number
        line_numbers = re.findall(r'^\d+\..*', data, re.MULTILINE)


        # Remove serial numbers using regular expression
        cleaned_strings = [ re.sub(r'^\d+\.\s+', '', s) for s in line_numbers]

        # Remove empty strings
        non_empty_strings = [s for s in cleaned_strings if s]

        # Remove duplicates by converting the list to a set and back to a list
        non_empty_strings = list(set(non_empty_strings))
        
        df_locations_new=pd.DataFrame()
        df_locations_new=get_df_locations(non_empty_strings,coordinates)
        
        df_locations_new['location_id'] = df_locations_new['location_id'].astype(int)
        df_locations_new['address_obj'] = df_locations_new['address_obj'].astype(str)
        # df_locations_new['category']='Restaurant'
        df_locations_new=df_locations_new.drop_duplicates(subset='location_id')
        
        df_location_existing = pd.read_sql_table('locations',engine_p)
        df_locations_new=df_locations_new[~df_locations_new['location_id'].isin(df_location_existing['location_id'])]

        df_locations_new = df_locations_new.reset_index(drop=True)
        df_locations_new.info()
        
        # Save DataFrame to SQL database
        # df.to_sql("Locations",engine_p,if_exists='append',index=False)
        df_locations_new.to_sql("locations",engine_mysql,if_exists='append',index=False)


        # print("Data has been retrieved and saved to the database.")
        # non_empty_strings


NameError: name 'pd' is not defined